In [2]:
from keras.models import Sequential
from keras.layers import Dense,Activation,LSTM

from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys




Using TensorFlow backend.


In [3]:
path = "./kazeno_matasaburo.txt"
bindata = open(path, "rb").read()
text = bindata.decode("shift_jis")

In [4]:
print("Size of text: ", len(text))

Size of text:  33106


In [5]:
chars = sorted(list(set(text)))

In [9]:
char_indices = dict((c,i) for i,c in enumerate(chars))

In [11]:
indices_char = dict((i,c) for i,c in enumerate(chars))

In [14]:
maxlen = 40
step = 3
sentences = []
next_chars = []

In [15]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])


In [19]:
print('Number of sentences: ', len(sentences))

Number of sentences:  11022


In [22]:
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)

In [24]:
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [31]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i,char_indices[next_chars[i]]] = 1

In [35]:
#モデル定義
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))

In [36]:
model.add(Dense(len(chars)))

In [37]:
model.add(Activation('softmax')) #0~1に値を変換
optimizer = RMSprop(lr=0.01)

In [38]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [40]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probs = np.random.multinomial(1,preds, 1)
    return np.argmax(probs)

In [41]:
for iteration in range(1,60):
    print()
    print('-' * 50)
    print('繰り返し回数:', iteration)
    model.fit(X, y, batch_size=128, epochs=1)
    
start_index = random.randint(0,len(text) - maxlen -1)

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----diversity' + diversity)
    
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('---- Seedを生成しました: ' + sentence )
    sys.stdout.write(generated)
    
    
    for i in range(400):
        x = np.zeros((1,maxlen,len(chars)))
        for t, char in enumerate(sentence):
            x[0,t,char_indices[char]] = 1.
            
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        
        generated += next_char
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flash()
    print()


--------------------------------------------------
繰り返し回数: 1


Epoch 1/1


  128/11022 [..............................] - ETA: 1:11 - loss: 6.7259

  256/11022 [..............................] - ETA: 53s - loss: 6.6651 

  384/11022 [>.............................] - ETA: 48s - loss: 6.1502

...

 1024/11022 [=>............................] - ETA: 34s - loss: 1.0293

 1152/11022 [==>...........................] - ETA: 33s - loss: 1.0329

 1280/11022 [==>...........................] - ETA: 33s - loss: 1.0255

 1408/11022 [==>...........................] - ETA: 33s - loss: 1.0391

KeyboardInterrupt: 